In [22]:
from hanziconv import HanziConv

In [32]:
f = open(r'E:\SHARON\NLP\AA\wiki_00.txt',encoding='UTF-8')
txt = f.read()
s_txt = HanziConv.toSimplified(txt)

In [40]:
s_txt[1:100]

'doc id="13" url="https://zh.wikipedia.org/wiki?curid=13" title="数学">\n数学\n\n数学是利用符号语言研究数量、结构、变化以及空间等概念'

In [35]:
with open(r'E:\SHARON\NLP\AA\wiki_00_s.txt',"w",encoding='UTF-8') as f1:
    f1.write(s_txt)

In [36]:
import re
def token(string):
    return ' '.join(re.findall('[\w\d]+',string))

In [41]:
all_articles = token(s_txt)

In [44]:
len(all_articles)

419049216

In [48]:
import jieba
def cut(string):return list(jieba.cut(string))
ALL_TOKENS = cut(all_articles[0:10000000])

In [51]:
valid_tokens = [t for t in ALL_TOKENS if t.strip() and t!='n']

In [72]:
from collections import Counter
word_count = Counter(valid_tokens)

In [73]:
len(word_count.most_common())

231499

In [74]:
frequency_all = [f for w,f in word_count.most_common()]

In [75]:
frequency_sum = sum(frequency_all)

In [76]:
frequency_sum

4667763

In [78]:
def get_prob(word):
    esp = 1/frequency_sum
    if word in word_count:
        return word_count[word]/frequency_sum
    else:
        return esp

In [82]:
from functools import reduce
def product(numbers):
    return reduce(lambda n1,n2:n1*n2, numbers)

In [83]:
def language_model_one_gram(string):
    words = cut(string)
    return product([get_prob(w) for w in words])

In [86]:
language_model_one_gram('下个月举办广交会')

2.871153818943975e-17

In [87]:
all_2_gram_words = [''.join(valid_tokens[i:i+2]) for i in range(len(valid_tokens[:-2]))]
_2_gram_sum = len(all_2_gram_words)
_2_gram_counter = Counter(all_2_gram_words)
def get_combination_prob(w1,w2):
    if w1+w2 in _2_gram_counter: return _2_gram_counter[w1+w2]/_2_gram_sum
    else:
        return 1/_2_gram_sum

In [88]:
def get_prob_2_gram(w1, w2):
    return get_combination_prob(w1,w2)/get_prob(w1)

In [89]:
def language_model_two_gram(string):
    string_prob = 1
    words = cut(string)
    for i, word in enumerate(words):
        if i == 0:
            prob = get_prob(word)
        else:
            previous = words[i-1]
            prob = get_prob_2_gram(previous, word)
        string_prob *= prob
    return string_prob

In [97]:
language_model_two_gram('一个苹果掉在了牛顿头上')

2.713159860091762e-19

In [98]:
language_model_two_gram('小明是苹果')

4.314568168934126e-12

In [100]:
cut('这个房间的空调用的是风机盘管系统')

['这个', '房间', '的', '空调', '用', '的', '是', '风机', '盘管', '系统']